<a href="https://colab.research.google.com/github/momchilgeorgiev/2023DL/blob/main/Testing_this_time_it_works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Flatten, Conv2D, \
                             Dropout, Input, MaxPool2D, Rescaling, BatchNormalization, \
                             GlobalAveragePooling2D, Concatenate, Reshape, Input, UpSampling2D

from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential, Model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def cmm(model, train_data, epochs = 2):
  model.compile(optimizer = 'adam', loss = 'sparse_classifiacation_crossentropy', metrics = ['accuracy'])
  model.fit(train_data, epochs = 2)

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

In [3]:
(ds_train, ds_test), ds_info = tfds.load(
    'cifar100',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar100/3.0.2.incompleteQILK6B/cifar100-train.tfrecord*...:   0%|        …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar100/3.0.2.incompleteQILK6B/cifar100-test.tfrecord*...:   0%|         …

Dataset cifar100 downloaded and prepared to /root/tensorflow_datasets/cifar100/3.0.2. Subsequent calls will reuse this data.


In [4]:
ds_train.element_spec

(TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [ ]:
ds_info.features

FeaturesDict({
    'coarse_label': ClassLabel(shape=(), dtype=int64, num_classes=20),
    'id': Text(shape=(), dtype=string),
    'image': Image(shape=(32, 32, 3), dtype=uint8),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=100),
})

### Dataset to numpy:

### Preprocessing #2

In [5]:
def preprocess(image, label):
    image = tf.image.per_image_standardization(image)

    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    return image, label

In [4]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 16
NUM_CLASSES = ds_info.features['label'].num_classes

ds_train = ds_train.cache().shuffle(1000).batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTO)

ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [7]:
ds_train.element_spec

(TensorSpec(shape=(None, 32, 32, 3), dtype=tf.uint8, name=None),
 TensorSpec(shape=(None,), dtype=tf.int64, name=None))

In [5]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_input_VGG

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_Resnet50

from tensorflow.keras.applications.densenet import DenseNet121

In [6]:
tf.keras.backend.clear_session()

In [7]:
# Load the VGG16 model with ImageNet weights
vgg16 = VGG16(weights='imagenet', include_top=False)

# Load the DenseNet121 model with ImageNet weights
densenet121 = DenseNet121(weights='imagenet', include_top=False)

vgg16.trainable = False
densenet121.trainable = False
# Reshape the output of VGG16 to match the input shape of DenseNet121
# Assuming a common denominator shape for both
vgg16_output = vgg16.output
reshape_layer = Reshape((32, 32, 3))(vgg16_output)

# Add the reshaped output of VGG16 as input to DenseNet121
densenet121_input = densenet121(reshape_layer)

29084464/29084464 [==============================] - 1s 0us/step


In [8]:
model_a = Model(inputs = vgg16.layers[0].input, outputs = vgg16.layers[-1].output)

In [9]:
model_b = Model(inputs = densenet121.layers[3].input, outputs = densenet121.layers[-2].output)

In [57]:
model_b.input

<KerasTensor: shape=(None, None, None, 64) dtype=float32 (created by layer 'input_8')>

In [55]:
model_a.output

<KerasTensor: shape=(None, None, None, 512) dtype=float32 (created by layer 'block5_pool')>

In [21]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Input

bridge_conv = Conv2D(64, (1,1), padding='same')(model_a.output)
bridge_bn = BatchNormalization()(bridge_conv)
bridge_activation = Activation('relu')(bridge_bn)

In [22]:
densenet121_input = model_b(bridge_activation)

In [23]:
combined_model = Model(inputs=model_a.input, outputs=densenet121_input)

In [ ]:
plot_model(model = combined_model, show_shapes = True)

In [32]:
cmm(combined_model, ds_train)

Epoch 1/2


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer 'pool2_pool' (type AveragePooling2D).
    
    Negative dimension size caused by subtracting 2 from 1 for '{{n